## BÁO CÁO BÀI TẬP LỚN HỌC MÁY CUỐI KỲ

Mã Lớp: INT3405_1

Họ và tên: Nguyễn Trọng Hải

Mã số sinh viên: 18020447

## MÔ TẢ BÀI TOÁN
Quora là một nền tảng cho phép mọi người học hỏi lẫn nhau. Trên Quora, mọi người có thể đặt câu hỏi và kết nối với những người khác, những người đóng góp thông tin chi tiết độc đáo và câu trả lời chất lượng. 

Một thách thức quan trọng là loại bỏ những câu hỏi thiếu chân thành - những câu hỏi được đặt ra dựa trên những tiền đề sai lầm hoặc có ý định đưa ra một tuyên bố hơn là tìm kiếm những câu trả lời hữu ích.

Trong cuộc thi này, chúng ta sẽ phát triển các mô hình xác định và gắn cờ cho các câu hỏi không chân thành.

* Input: Các câu hỏi trên Quora dưới dạng text
* Output: giá trị 0 hoặc 1 (0: câu hỏi chân thành; 1: câu hỏi không chân thành)

## PHÂN TÍCH DỮ LIỆU


#### Import thư viện cần thiết

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.initializers import *
from tensorflow.keras.optimizers import *
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import *

import warnings
warnings.filterwarnings("ignore")

import os
import time
import gc
import re
import glob

#### Đọc dữ liệu

In [ ]:
train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')

In [ ]:
train.info()

In [ ]:
test.info()

***Nhận xét***: Dữ liệu huấn luyện và kiểm thử không có giá trị null

In [ ]:
ax, fig = plt.subplots(figsize=(10, 7))
question_class = train["target"].value_counts()
question_class.plot(kind= 'bar', color= ["blue", "orange"])
plt.title('Bar chart')
plt.show()

In [ ]:
print("Tỉ lệ phần trăm số câu hỏi Insincere là:", (len(train.loc[train.target==1])) / (len(train.loc[train.target == 0])) * 100)

Nhận xét về dữ liệu
Dựa vào biểu đồ trên ta có thể thấy, số lượng câu hỏi gán nhãn chân thành chiếm phần lớn với tỷ lệ 93.4% còn những câu hỏi không chân thành chỉ chiếm 6.6%

=> Tập dữ liệu bị mất cân bằng, vì thế nên khi đánh giá chất lượng mô hình ta không nên lựa chọn độ chính xác(accuracy) làm chỉ số đánh giá mô hình. Thay vào đó ta có thể sử dụng các metric thay thế như: F1_score, Recall,..

#### Phân tích từng câu hỏi

**Số lượng từ trong câu**

In [ ]:
words = train['question_text'].apply(lambda x: len(x) - len(''.join(x.split())) + 1)
train['words'] = words
words = train.loc[train['words']<200]['words']
sns.distplot(words, color='g')
plt.show()

In [ ]:
print('Số lượng từ trung bình của các câu hỏi trong dữ liệu huấn luyện là {0:.0f}.'.format(np.mean(train['question_text'].apply(lambda x: len(x.split())))))
print('Số lượng từ trung bình của các câu hỏi trong dữ liệu kiểm thử là {0:.0f}.'.format(np.mean(test['question_text'].apply(lambda x: len(x.split())))))

In [ ]:
print('Số lượng từ lớn nhất của các câu hỏi trong dữ liệu huấn luyện là {0:.0f}.'.format(np.max(train['question_text'].apply(lambda x: len(x.split())))))
print('Số lượng từ lớn nhất của các câu hỏi trong dữ liệu kiểm thử là {0:.0f}.'.format(np.max(test['question_text'].apply(lambda x: len(x.split())))))

In [ ]:
print('Số lượng ký tự trung bình của các câu hỏi trong dữ liệu huấn luyện là {0:.0f}.'.format(np.mean(train['question_text'].apply(lambda x: len(x)))))
print('Số lượng ký tự trung bình của các câu hỏi trong dữ liệu kiểm thử là {0:.0f}.'.format(np.mean(test['question_text'].apply(lambda x: len(x)))))

***Nhận xét:*** Có thể thấy độ dài trung bình của các câu hỏi trong tập dữ liệu huấn luyện và kiểm thử tương tự nhau, tuy nhiên có những câu hỏi khá dài trong tập dữ liệu huấn luyện

### Xử LÝ DỮ LIỆU

#### CÁC BƯỚC TIỀN XỬ LÝ DỮ LIỆU:

* Lowering
* Xoá các số (do các số không có ý nghĩa trong việc phân lớp)
* Xoá các kí tự đặc biệt
* Mispelling (thay thế các từ viết tắt)

Như đã phân tích về dữ liệu ở trên, ta sẽ quy chuẩn số lượng từ trong một câu hỏi. Do ta sử dụng tập từ điển đã được huấn luyện sẵn là tập embeddings nên mỗi vector có độ dài là 300, và mỗi câu hỏi sẽ có tối đa 80 từ

In [ ]:
embed_size = 300 #độ dài của vector
max_features = 120000 #số lượng từ xuất hiện nhiều nhất để huấn luyện
maxlen = 80 #số lượng từ trong câu 

**Lowering**

In [ ]:
train["question_text"] = train["question_text"].apply(lambda x: x.lower())
test["question_text"] = test["question_text"].apply(lambda x: x.lower())

**Xoá các số**

In [ ]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
train["question_text"] = train["question_text"].progress_apply(lambda x: clean_numbers(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_numbers(x))

**Xoá ký tự đặc biệt**

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

In [ ]:
train["question_text"] = train["question_text"].progress_apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_text(x))

**Mispelling**

In [ ]:
mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [ ]:
train["question_text"] = train["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
test["question_text"] = test["question_text"].apply(lambda x: replace_typical_misspell(x))

**Chia dữ liệu để huấn luyện với tỉ lệ 90:10**

In [ ]:
# split to train and val
train, val = train_test_split(train, test_size=0.1, random_state=2018) 

In [ ]:
# fill up the missing values
train_X = train["question_text"].fillna("_na_").values
val_X = val["question_text"].fillna("_na_").values
test_X = test["question_text"].fillna("_##_").values

In [ ]:
# Get the target values
train_y = train['target'].values
val_y = val['target'].values 

**Tokenize**

Tokenize các câu hỏi thành các token(các từ) sau đó gán chỉ số cho từng từ và thay từng từ thành chỉ số tương ứng của từ đó.

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X)) # gán chỉ số cho từng từ
word_index = tokenizer.word_index # lấy chỉ số của các từ
train_X = tokenizer.texts_to_sequences(train_X) # thay thế các từ bởi chỉ số tương ứng của từ đó
val_X = tokenizer.texts_to_sequences(val_X)  # thay thế các từ bởi chỉ số tương ứng của từ đó
test_X = tokenizer.texts_to_sequences(test_X) # thay thế các từ bởi chỉ số tương ứng của từ đó

**Pad chuỗi** - nếu số lượng từ trong câu hỏi lớn hơn 'max_len' thì chuyển thành 'max_len' hoặc nếu số từ trong văn bản ít hơn 'max_len' thì bổ sung thêm số 0 vào các giá trị còn lại

In [ ]:
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

**Tráo dữ liệu**

In [ ]:
#shuffling the data
np.random.seed(1203)
trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))

train_X = train_X[trn_idx]
val_X = val_X[val_idx]
train_y = train_y[trn_idx]
val_y = val_y[val_idx]   

Ở Bài toán này, để tăng hiệu quả phân lớp, ta sẽ sử dụng từ điển chứa trọng số của các từ đã được huấn luyện sẵn (Glove)

In [ ]:
! unzip ../input/quora-insincere-questions-classification/embeddings.zip -d input/

Ta sẽ tạo ma trận embedding chỉ chứa những từ trong ma trận word2vec mà có trong word_index

In [ ]:
EMBEDDING_FILE = './input/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE)) # lấy hệ số của từng từ trong tập embedding

all_embs = np.stack(embeddings_index.values()) 
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

# word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size)) # tạo hệ số ma trận ngẫu nhiên 
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word) 
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector # thay các hệ số của những từ có trong tập embedding

## HUẤN LUYỆN MÔ HÌNH

#### Định nghĩa F1-score

Do dữ liệu bị mất cân bằng nên metric phù hợp nhất là F1 score

In [ ]:
!pip install tensorflow-addons

In [ ]:
f1 = tfa.metrics.F1Score(num_classes=1, threshold=0.5)
accuracy = tf.keras.metrics.BinaryAccuracy(
    name='binary_accuracy', dtype=None, threshold=0.5
)

#### Định nghĩa hàm Learning rate và Early stopping để giúp hàm loss hội tụ hiệu quả hơn

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.0001, verbose=0)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=1, mode='auto', restore_best_weights=True)
callbacks = [reduce_lr, earlystopping]

#### Thiết lập model

Xây dựng mô hình huấn luyện bằng LSTM (Long Short-Term Memory Networks)

Trước khi hiểu mô hình LSTM là gì thì ta cần hiểu qua về mạng RNN - recurrent neural network. RNN có thể mang thông tin từ các layer trước đến layer sau, nên nó có thể dùng để xử lý thông tin dạng chuỗi. Một ví dụ của RNN trong bài toán dự đoán video, RNN có thể mang thông tin của frame ảnh từ state trước tới state sau, tuy nhiên state ở trước đó càng xa thì càng bị vanishing gradient, nghĩa là thông tin chỉ mang được qua một lượng state nhất định hay nói cách khác là model chỉ học được từ các state gần nó - short term memory

Vì vậy, mô hình Long short term memory ra đời để giải quyết vấn đề khi ta cần các thông tin từ state ở trước đó rất xa và tránh được vanishing gradient. Nó vẫn giữ tư tưởng chính của RNN là sự sao chép kiến trúc theo dạng chuỗi nhưng có phần phức tạp hơn.

In [ ]:
def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
    return model

#### Train model

In [ ]:
model = model_lstm_atten(embedding_matrix)
model.summary()

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=10, validation_data=(val_X, val_y), callbacks=callbacks)

#### Predict

In [ ]:
pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)
pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)

#### Xác định threshold có F1 cao nhất với threshold trong khoảng từ 0.10 đến 0.50

In [ ]:
def f1_smart(y_true, y_pred):
    thresholds = []
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        res = f1_score(y_true, (y_pred > thresh).astype(int))
        thresholds.append([thresh, res])
        print("F1 score at threshold {0} is {1}".format(thresh, res))

    thresholds.sort(key=lambda x: x[1], reverse=True)
    best_thresh = thresholds[0][0]
    best_f1 = thresholds[0][1]
    print("Best threshold: ", best_thresh)
    return  best_f1, best_thresh

In [ ]:
f1, threshold = f1_smart(val_y, pred_val_y)
print('Optimal F1: {} at threshold: {}'.format(f1, threshold))

## SUBMISSION

In [ ]:
pred_test_y = (pred_test_y >threshold).astype(int)
out_df = pd.DataFrame({"qid":test["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)